In [1]:
import sys
import os

# プロジェクトのルートディレクトリを指定
project_root = os.path.abspath(os.path.join(os.getcwd(), './..'))
sys.path.append(project_root)

current_dir = os.getcwd()
os.environ['PROJECT_ROOT'] = os.path.join(current_dir, '..')

In [2]:
from omegaconf import OmegaConf
from config.modifier import dynamically_modify_train_config
config_paths = [
        '../config/dataset/gen1/event_frame/sequence/base.yaml',
        '../config/model/rvt_ssm/rvt_ssm_frame.yaml',
        '../config/experiment/sequence/train.yaml',
    ]

configs = [OmegaConf.load(path) for path in config_paths]
merged_conf = OmegaConf.merge(*configs)
dynamically_modify_train_config(config=merged_conf)

num_class 2
Set partition sizes: (8, 10)


In [3]:
from modules.fetch import fetch_data_module, fetch_model_module

data = fetch_data_module(merged_conf)
data.setup('fit')
model = fetch_model_module(merged_conf)
model.setup('fit')

train dataset size: 49
valid dataset size: 58
rvt
RVT-SSM
PAFPN
neck input channels (64, 128, 256)
head strides (8, 16, 32)
YOLOX-Head


In [4]:
data_iter = iter(data.train_dataloader())


In [6]:
ssm_model = model.model
ssm_model.eval()

RVTYOLOX(
  (backbone): RVTSSM(
    (stages): ModuleList(
      (0): RVTSSMStage(
        (downsample_cf2cl): ConvDownsampling_Cf2Cl(
          (conv): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3), bias=False)
          (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        )
        (att_blocks): ModuleList(
          (0): MaxVitAttentionPairCl(
            (att_window): PartitionAttentionCl(
              (norm1): Identity()
              (self_attn): SelfAttentionCl(
                (qkv): Linear(in_features=32, out_features=96, bias=True)
                (proj): Linear(in_features=32, out_features=32, bias=True)
              )
              (ls1): LayerScale()
              (drop_path1): Identity()
              (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
              (mlp): MLP(
                (net): Sequential(
                  (0): Sequential(
                    (0): Linear(in_features=32, out_features=128, bias=True)
     

In [11]:
ssm_model.eval()

RVTYOLOX(
  (backbone): RVTSSM(
    (stages): ModuleList(
      (0): RVTSSMStage(
        (downsample_cf2cl): ConvDownsampling_Cf2Cl(
          (conv): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3), bias=False)
          (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        )
        (att_blocks): ModuleList(
          (0): MaxVitAttentionPairCl(
            (att_window): PartitionAttentionCl(
              (norm1): Identity()
              (self_attn): SelfAttentionCl(
                (qkv): Linear(in_features=32, out_features=96, bias=True)
                (proj): Linear(in_features=32, out_features=32, bias=True)
              )
              (ls1): LayerScale()
              (drop_path1): Identity()
              (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
              (mlp): MLP(
                (net): Sequential(
                  (0): Sequential(
                    (0): Linear(in_features=32, out_features=128, bias=True)
     

In [12]:
from utils.yolox_utils import to_yolox
batch = next(data_iter)
sequence_events = batch['events'].float()  # [batch, sequence_len, ch, h, w]
sequence_labels = batch['labels']  # [batch, sequence_len, num, bbox]
sequence_timestamps = batch['timestamps']
is_first_sample = batch['is_start_sequence']  # list[] * batch
sequence_is_padded_mask = batch['mask']  # [batch, sequence_len]

sequence_events = sequence_events.permute(1, 0, 2, 3, 4)  # -> [L, B, C, H, W]

sequence_targets = to_yolox(sequence_labels, mode='val').float()

prev_state =None

In [13]:
outputs, state = ssm_model(sequence_events, prev_state)
prev_state = state

: 